## Import Liberares

In [4]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.signal import butter, filtfilt
from mediapipe.python.solutions.pose import POSE_CONNECTIONS

## Butterworth filter functions

Here we use **Butterworth filters** via Scipy library. [signal.butter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html) is a function provided by the [scipy.signal](https://docs.scipy.org/doc/scipy/reference/signal.html) module in Python. It is used to design digital Butterworth filters, which are a type of infinite impulse response (IIR) filter commonly used for signal processing tasks, including filtering EEG signals. The Butterworth filter is characterized by having a flat frequency response in its passband and a gradual roll-off in its stopband.

In [1]:
def butter_lowpass(cutoff, fs, order=2):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq   # normalized cutoff frequency
    b, a = butter(order, normal_cutoff, btype = "low", analog = False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=2):
    b, a = butter_lowpass(cutoff, fs, order=order)
    return filtfilt(b, a, data)
    

## Kalman filter function
Here we use a simple 1D **Kalman Filter** implemented in Python. The KalmanFilter1D class models the process and measurement uncertainties and continuously updates its estimate as new data arrives. The Kalman Filter is an optimal recursive algorithm that combines prior predictions with new noisy measurements to produce a more accurate estimate of the true value. It is widely used in signal processing, control systems, and sensor fusion due to its ability to reduce noise and track system states over time.

### Kalman Filter 1D - Key Formulas

#### 1. Prior (Prediction) Estimate
The prior (predicted) estimate is the prediction of the current state before seeing the measurement:

$$
x_{\text{prior}} = x_{\text{posteri}}
$$

#### 2. Prior Error Estimate
The uncertainty of the prior estimate increases due to process noise:

$$
P_{\text{prior}} = P_{\text{posteri}} + Q
$$

Where:  
- P_prior is the predicted (prior) error estimate  
- P_posteri is the previous posterior error estimate  
- Q is the process noise variance

#### 3. Kalman Gain
The Kalman Gain determines how much we trust the measurement vs. the prediction:

$$
K = \frac{P_{\text{prior}}}{P_{\text{prior}} + R}
$$

Where:  
- K is the Kalman Gain  
- R is the measurement noise variance

#### 4. Posterior (Updated) Estimate
Update the estimate with the new measurement:

$$
x_{\text{posteri}} = x_{\text{prior}} + K \cdot (z - x_{\text{prior}})
$$

Where:  
- x_posteri is the updated (posterior) estimate  
- z is the new measurement

#### 5. Posterior Error Estimate
Update the error covariance after measurement:

$$
P_{\text{posteri}} = (1 - K) \cdot P_{\text{prior}}
$$

Where:  
- P_posteri is the updated error estimate



In [1]:
class KalmanFilter1D:
    def __init__(self, process_variance=0.001, measurement_variance=0.01):
        self.process_variance = process_variance
        self.measurement_variance = measurement_variance
        self.posteri_estimate = 0.0
        self.posteri_error_estimate = 1.0

    def update(self, measurement):
        priori_estimate = self.posteri_estimate
        priori_error_estimate = self.posteri_error_estimate + self.process_variance
        blending_factor = priori_error_estimate / (priori_error_estimate + self.measurement_variance)
        self.posteri_estimate = priori_estimate + blending_factor * (measurement - priori_estimate)
        self.posteri_error_estimate = (1 - blending_factor) * priori_error_estimate
        return self.posteri_estimate